In [73]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime

In [74]:
year = 2021
cal = 2 # 1 for WT, 2 for UCI, etc.
results_dict = {}
stages_dict = {}

url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')

if year == datetime.datetime.today().year:
    tab = soup.find_all('table', class_='sortTabell tablesorter')[1]
else:
    tab = soup.find_all('table', class_='sortTabell tablesorter')[0]

refs = [i['href'] for i in tab.find_all('a', href=True)]
races_raw = [x for x in refs if 'race' in x]
races = [z for z in races_raw if 'k' not in z]
urls = ['https://firstcycling.com/'+y for y in races]
for race in urls:
    r_race = requests.get(race)
    soup_race = BeautifulSoup(r_race.text,'html.parser')
    race_name = soup_race.find('h1').text
    rows = soup_race.find_all('tbody')[0].find_all('tr')
    result = []
    for tr in rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            result.append(row)
            
    stage_soup = BeautifulSoup(requests.get(race+'&k=2').text,'html.parser') 
    try:
        if stage_soup.find('h1', class_='delinfo').text == 'Stages':
            stg_ref = [i['href'] for i in stage_soup.find_all('table', class_="tablesorter")[1].find_all('a')]
            stages = [x for x in stg_ref if 'race' in x]
            stages_url = ['https://firstcycling.com/'+y for y in stages]
            for stage in stages_url:
                r_stage = requests.get(stage)
                soup_stage = BeautifulSoup(r_stage.text,'html.parser')
                for i in soup_stage.find_all('select')[1].find_all('option'):
                    if i.has_attr('selected'):
                        stage_name = race_name+' stage '+i.text
                rows_stage = soup_stage.find_all('tbody')[0].find_all('tr')
                result_stage = []
                for tr in rows_stage:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td if tr.text.strip()]
                    if row:
                        result_stage.append(row)
                stages_dict[stage_name] = pd.DataFrame(result_stage,
                                                       columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
    except:
        continue
            
            
    try:        
        results_dict[race_name] = pd.DataFrame(result, 
                                               columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
    except ValueError:
        continue

In [154]:
print(results_dict.keys())

dict_keys(['Le Samyn - 2021', 'CC Africa ITT - 2021', 'Umag Trophy - 2021', 'Trofeo Laigueglia - 2021', 'CC Africa RR - 2021', 'Grand Prix Mediterrennean - 2021', 'Strade Bianche - 2021', 'Grand Prix Gündogmus - 2021', 'Porec Trophy - 2021', 'GP Jean-Pierre Monseré - 2021', 'GP Industria & Artigianato - 2021', 'Paris - Nice - 2021', 'Tirreno - Adriatico - 2021', 'Istrian Spring Trophy - 2021', 'Paris - Troyes - 2021', 'Nokere Koerse - 2021', 'Bredene Koksijde Classic - 2021', 'Milano - Sanremo - 2021', 'GP Slovenian Istria - 2021', 'Per Sempre Alfredo - 2021'])


In [153]:
temp = results_dict['Paris - Nice - 2021']['Total time'] ##take the total time of the race (it is a string)

temp_list = []
for x in temp:
    if x is not None:
        y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
        for i in range(0, len(y)):
            y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
        temp_int = 0
        for j in range(len(y)-1, -1, -1):
            temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
        temp_list.append(temp_int)                  ## appends them to list

for k in range(len(temp_list), len(temp)):
    temp_list.append('None')                        ## appends 'None' to list before merging into DF
results_dict['Paris - Nice - 2021']['Time seconds'] = temp_list

results_dict['Paris - Nice - 2021']['Perc of time'] = ""  ## creates empty column for percentage of time

for index in range(1, len(results_dict['Paris - Nice - 2021']['Time seconds'])):      ## sums timegaps and total time of the winner
    if results_dict['Paris - Nice - 2021']['Time seconds'][index] != 'None':
        results_dict['Paris - Nice - 2021']['Time seconds'][index] +=  int(results_dict['Paris - Nice - 2021']['Time seconds'][0])

for index  in range(len(results_dict['Paris - Nice - 2021']['Time seconds'])) :
    if results_dict['Paris - Nice - 2021']['Time seconds'][index] != 'None':
        results_dict['Paris - Nice - 2021']['Perc of time'][index] = int(results_dict['Paris - Nice - 2021']['Time seconds'][index]) / int(results_dict['Paris - Nice - 2021']['Time seconds'][0])
    else:
        results_dict['Paris - Nice - 2021']['Perc of time'][index] = 'None'

    
print(results_dict['Paris - Nice - 2021'])

    Position                  Rider  Born                  Team Total time  \
0         01  Maximilian Schachmann  1994        BORA-hansgrohe   28:49:51   
1         02       Aleksandr Vlasov  1996   Astana-Premier Tech       + 19   
2         03           Ion Izagirre  1989   Astana-Premier Tech       + 23   
3         04         Lucas Hamilton  1996     Team BikeExchange       + 41   
4         05           Tiesj Benoot  1994              Team DSM       + 42   
..       ...                    ...   ...                   ...        ...   
156      DNF          Max Walscheid  1993         Qhubeka ASSOS       None   
157      DNF    Christopher Lawless  1995  Total Direct Energie       None   
158      DNF       Dries van Gestel  1994  Total Direct Energie       None   
159      DNF      Alexis Vuillermoz  1988  Total Direct Energie       None   
160      DNF        Brandon McNulty  1998     UAE Team Emirates       None   

    FC points Time seconds Perc of time  
0         500       1

In [156]:
for name in results_dict.keys():
    temp = results_dict[name]['Total time'] ##take the total time of the race (it is a string)

    temp_list = []
    for x in temp:
        if x is not None:
            y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
            for i in range(0, len(y)):
                y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
            temp_int = 0
            for j in range(len(y)-1, -1, -1):
                temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
            temp_list.append(temp_int)                  ## appends them to list

    for k in range(len(temp_list), len(temp)):
        temp_list.append('None')                        ## appends 'None' to list before merging into DF
    results_dict[name]['Time seconds'] = temp_list

    results_dict[name]['Perc of time'] = ""  ## creates empty column for percentage of time

    for index in range(1, len(results_dict[name]['Time seconds'])):      ## sums timegaps and total time of the winner
        if results_dict[name]['Time seconds'][index] != 'None':
            results_dict[name]['Time seconds'][index] +=  int(results_dict[name]['Time seconds'][0])

    for index  in range(len(results_dict[name]['Time seconds'])) :
        if results_dict[name]['Time seconds'][index] != 'None':
            results_dict[name]['Perc of time'][index] = int(results_dict[name]['Time seconds'][index]) / int(results_dict[name]['Time seconds'][0])
        else:
            results_dict[name]['Perc of time'][index] = 'None'

        
    print(results_dict[name])
    

    Position                    Rider  Born                      Team  \
0         01              Tim Merlier  1992             Alpecin-Fenix   
1         02            Rasmus Tiller  1996    Uno-X Pro Cycling Team   
2         03         Andrea Pasqualon  1988  Intermarché-Wanty-Gobert   
3         04            Sep Vanmarcke  1988    Israel Start-Up Nation   
4         05          Hugo Hofstetter  1994    Israel Start-Up Nation   
..       ...                      ...   ...                       ...   
165      DNF    Julien Van Den Brande  1995       Tarteletto - Isorex   
166      DNF             Enzo Wouters  1996       Tarteletto - Isorex   
167      DNF          Donavan Grondin  2000              Arkéa-Samsic   
168      DNF      Christopher Lawless  1995      Total Direct Energie   
169      DNF  Jonas Iversby Hvideberg  1999    Uno-X Pro Cycling Team   

    Total time FC points Time seconds Perc of time  
0      4:34:29       125        16469          1.0  
1         + 00   

In [159]:
for name in stages_dict.keys():
    temp = stages_dict[name]['Total time'] ##take the total time of the race (it is a string)

    temp_list = []
    for x in temp:
        if x is not None:
            y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
            for i in range(0, len(y)):
                y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
            temp_int = 0
            for j in range(len(y)-1, -1, -1):
                temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
            temp_list.append(temp_int)                  ## appends them to list

    for k in range(len(temp_list), len(temp)):
        temp_list.append('None')                        ## appends 'None' to list before merging into DF
    stages_dict[name]['Time seconds'] = temp_list

    stages_dict[name]['Perc of time'] = ""  ## creates empty column for percentage of time

    for index in range(1, len(stages_dict[name]['Time seconds'])):      ## sums timegaps and total time of the winner
        if stages_dict[name]['Time seconds'][index] != 'None':
            stages_dict[name]['Time seconds'][index] +=  int(stages_dict[name]['Time seconds'][0])

    for index  in range(len(stages_dict[name]['Time seconds'])) :
        if stages_dict[name]['Time seconds'][index] != 'None':
            stages_dict[name]['Perc of time'][index] = int(stages_dict[name]['Time seconds'][index]) / int(stages_dict[name]['Time seconds'][0])
        else:
            stages_dict[name]['Perc of time'][index] = 'None'

        
    print(stages_dict[name])
    

    Position                Rider  Born                   Team Total time  \
0         01          Sam Bennett  1990  Deceuninck-Quick Step   03:51:38   
1         02        Arnaud Démare  1991           Groupama-FDJ       + 00   
2         03        Mads Pedersen  1995         Trek-Segafredo       + 00   
3         04     Jasper Philipsen  1998          Alpecin-Fenix       + 00   
4         05        Bryan Coquard  1992     B&B Hotels p/b KTM       + 00   
..       ...                  ...   ...                    ...        ...   
156      157      Matteo Pelucchi  1989          Qhubeka ASSOS    + 08:38   
157      158         Kaden Groves  1998      Team BikeExchange    + 08:42   
158      159  Christopher Lawless  1995   Total Direct Energie    + 14:00   
159      160         Maxime Bouet  1986           Arkéa-Samsic    + 14:00   
160      DNF         Richie Porte  1985       INEOS Grenadiers       None   

    FC points Time seconds Perc of time  
0          60        13898       